# Tratando DFs

#### function: NaN_rows( )

In [ ]:
# Cria função para visualização rápida de rows específicas que contém Nulls em colunas específicas
''' DOC:
NaN_rows(df, col) recebe como argumentos um dataframe (var) e uma coluna específica (str).
Cria uma máscara que retorna apenas as linhas com NaN na coluna indicada, sem eliminar outras colunas.
'''

def NaN_rows(df, col):
    mask = df[col].isna()
    NaN_rows = df[mask]
    return NaN_rows


#### function: make_filtered_df( )

In [ ]:
# Cria função para aplicar múltiplos filtros semelhantes em um DF e armazenar DFs filtrados em arquivos.xlsx
''' Function DOC:
make_filtered_df(col, filter_value, df_name, [Project_DFs_number, save_excel, save_path])

Gera os nomes e DFs filtrados dinamicamente, armazena o resultado no dict Filtered_DFs e opcionalmente salva os DFs em arquivos .xlsx

A função demanda como argumentos obrigatórios: 
  - str col: o nome de uma coluna
  - str filter_value: uma lista com valores de filtros
  - str df_name: o nome do DF conforme armazenado como key em um dict em Project_DFs
A função recebe como argumentos opcionais:
  - int Project_DFs_number: o índice de um dict em Project_DFs, por padrão definido em 3. [2] Raw_DFs; [3] Working_DFs; [4] Treated_DFs; [5] Filtered_DFs. [0] e [1] resultarão em erro.
  - Boolean save_excel:True salva o df em um arquivo .xlsx no cwd ou diretório indicado. Por padrão definido como False, não salva arquivo .xlsx
  - str save_path: indica diretório para salvar arquivo .xlsx com o novo DF. Por padrão determina o cwd com a função os.getcwd()

'''

def make_filtered_dfs(df_name, col, filter_values, dict_number = 3, save_excel=False, save_path = os.getcwd()) :

    # Cria novo nome para o novo df unindo o df original ao filtro indicado
    new_df_name = df_name + '_' + '_'.join(filter_values)
    new_df_name = new_df_name.replace(' ','_').replace('-','_')

    # Seleciona o df indicado em df_name no dicionário indicado
    df = Project_DFs[dict_number][df_name]

    # Cria novo df globalmente a partir de lista de filtros passada como argumento filter_values
    globals()[new_df_name] = df[df[col].isin(filter_values)]
    Filtered_DFs[new_df_name] = globals()[new_df_name]


    if save_excel == True :
      globals()[new_df_name].to_excel(f'{save_path}\\{new_df_name}.xlsx', index = False)

      return print(f'''
                    {new_df_name} gerado com sucesso.
                    Shape: {globals()[new_df_name].shape}
                    Filtered_DFs atualizado com sucesso.
                    {print(Filtered_DFs.keys())}
                    Arquivo {new_df_name}.xlsx salvo com sucesso em {save_path}
                  ''')
    else :
       return print(f'''
                    {new_df_name} gerado com sucesso.
                    Shape: {globals()[new_df_name].shape}
                    Filtered_DFs atualizado com sucesso.
                    {print(Filtered_DFs.keys())}
                  ''')
       

## Tratando raw_Areas_Risco_Domicilios e raw_Areas_Risco_Pessoas

In [ ]:
# Criando cópias de trabalho e armazenando em Working_DFs
df_Areas_Risco_Domic   = Raw_DFs['raw_Areas_de_Risco_Domicilios_RAW'].copy()
df_Areas_Risco_Pessoas = Raw_DFs['raw_Areas_de_Risco_Pessoas_RAW'].copy()
dict_update_line       = {
                         'df_Areas_Risco_Domic' : df_Areas_Risco_Domic, 
                         'df_Areas_Risco_Pessoas' : df_Areas_Risco_Pessoas
                         }
Working_DFs.update(dict_update_line)


In [ ]:
# Cria nova coluna 'Código Município Completo' no df_Areas_Risco com apenas os 7 primeiros dígitos de COD_BATER
# correspondentes aos códigos dos municípios IBGE
df_Areas_Risco_Domic['Código_Município'] = df_Areas_Risco_Domic['COD_BATER'].astype(str).str[:7]

# Adiciona nomes de municípios, UFs e regiões e códigos de municípios e UFs conforme padrão IBGE a partir da coluna 'Código_Município'
df_Areas_Risco_Domic = add_IBGE_std(df_Areas_Risco_Domic, 'Código_Município', 'all', True)

In [ ]:
# Repete procedimento acima para df_Area_Risco_Pessoas

df_Areas_Risco_Pessoas['Código_Município']  = df_Areas_Risco_Pessoas['COD_BATER'].astype(str).str[:7]
df_Areas_Risco_Pessoas = add_IBGE_std(df_Areas_Risco_Pessoas, 'Código_Município', 'all', True)


#### Verificando Nulos e padronizando colunas chave

In [ ]:
# Verificando nulos em df_Areas_Risco_Domic
df_Areas_Risco_Domic.isnull().sum()

# Buscando rows de nulos em nome_municpio
NaN_rows(df_Areas_Risco_Domic, 'nome_municipio')

#* O erro se deve a erros de digitação no código em COD_BATER
#* O código 2301308 provavelmente se refere à cidade Araripe no Ceará, cod:2301307
#* O código 2409102 provavelmente se refere à cidade Passa-e-Fica no RN, cod: 2409100

# Buscando resultados pelas cidades e códigos

df_Areas_Risco_Domic[df_Areas_Risco_Domic.eq("2301307").any(axis=1)]
#* 0 rows
df_Areas_Risco_Domic[df_Areas_Risco_Domic.eq("araripe").any(axis=1)]
#* 0 rows
df_Areas_Risco_Domic[df_Areas_Risco_Domic.eq("2409100").any(axis=1)]
#*  0 rows
df_Areas_Risco_Domic[df_Areas_Risco_Domic.eq("passa-e-fica").any(axis=1)]
#* 0 rows

df = df_Areas_Risco_Domic.copy()

# Atualizando valores dos códigos IBGE corretos
df.at[641, 'Código_Município' ] = '2301307'
df.at[864, 'Código_Município']  = '2409100'

# Adicionando novamente nomenclatura padrão de unidades geográficas conforme padrão dos 
# dicionários criados a partir de raw_DTB_BRASIL_MUNICIPIO com a função add_IBGE_std()

# Remove colunas para recriação conforme códigos corretos
drop_list = ['cod_municipio', 'nome_municipio','cod_UF','nome_UF','nome_mesorregiao','nome_microrregiao']
df.drop(drop_list, axis = 1, inplace = True)

# Recria as colunas a partir dos códigos corrigidos
df = add_IBGE_std(df, 'Código_Município', 'all', True)

# Checking
df.isnull().sum()
#* 0 Nulls

# Atualizando df_Areas_Risco_Domic com as mudanças
df_Areas_Risco_Domic = df


In [ ]:
# Verificando nulos em df_Areas_Risco_Pessoas
df_Areas_Risco_Pessoas.isnull().sum()

# Buscando rows de nulos em nome_municpio
NaN_rows(df_Areas_Risco_Pessoas, 'nome_municipio')

#* Os nulos se devem ao mesmo problema em df_Areas_Risco_Domic

# Repete procedimento acima para df_Areas_Risco_Pessoas

df = df_Areas_Risco_Pessoas.copy()

# Atualizando valores dos códigos IBGE corretos

df.at[641, 'Código_Município' ] = '2301307'
df.at[864, 'Código_Município']  = '2409100'


# Adicionando nomenclatura padrão de unidades geográficas conforme padrão dos 
# dicionários criados a partir de raw_DTB_BRASIL_MUNICIPIO com a função add_IBGE_std()

# Remove colunas para recriação conforme códigos corretos
drop_list = ['cod_municipio', 'nome_municipio','cod_UF','nome_UF','nome_mesorregiao','nome_microrregiao']
df.drop(drop_list, axis = 1, inplace = True)

# A configuração de add_IBGE_std() abaixo cria as colunas a partir dos códigos conhecidos
df = add_IBGE_std(df, 'Código_Município', 'all', True)

# Checking
df.isnull().sum()
#* 0 Nulls

# Atualizando df_Areas_Risco_Domic com as mudanças
df_Areas_Risco_Pessoas = df

In [ ]:
# Salvando DFs tratados no dict Treated_DFs

DF_Areas_Risco_Domic = df_Areas_Risco_Domic
DF_Areas_Risco_Pessoas = df_Areas_Risco_Pessoas

dict_update_line = {
                   'DF_Areas_Risco_Domic' : DF_Areas_Risco_Domic, 
                   'DF_Areas_Risco_Pessoas' : DF_Areas_Risco_Pessoas
                   }
Treated_DFs.update(dict_update_line)


#### Unindo dfs Area_Risco_Pessoas e Area_Risco_Munic em um único df

In [ ]:
# Realiza inner join com união em COD_BATER e modifica sufixos de duplicatas para facilitar a sua remoção
df_Areas_Risco_Domic_Pessoas = pd.merge(DF_Areas_Risco_Domic,
                                        DF_Areas_Risco_Pessoas, 
                                        how      = 'inner',
                                        on = 'COD_BATER',
                                        suffixes = ('', '_remove'))

# Deleta colunas excedentes criadas pelo merge()
df_Areas_Risco_Domic_Pessoas.drop([col for col in df_Areas_Risco_Domic_Pessoas.columns if 'remove' in col], axis = 1, inplace = True)

# Adiciona joined DF em Treated_DFs
DF_Areas_Risco_Domic_Pessoas = df_Areas_Risco_Domic_Pessoas.copy()
Treated_DFs['DF_Areas_Risco_Domic_Pessoas'] = DF_Areas_Risco_Domic_Pessoas


#### Criando DFs filtrados por unidade geográfica a partir de DF_Areas_Risco_Domic_Pessoas e exportando DFs filtrados em .xlsx

In [ ]:
# Preparando listas para criar e salvar os dfs necessários com make_filtered_df() em loop

# Define diretório para armazenar os arquivos gerados pelos novos dfs
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Áreas de Risco'

# Criando os filtros com os valores desejados em cada escala territorial relevante para o projeto
Estados    = ['alagoas', 'bahia', 'pernambuco']

MicroRegs  = ['maceio', 'itamaraca', 'recife', 'suape', 'porto-seguro', 'ilheus-itabuna']

MesoRegs   = ['leste-alagoano', 'sul-baiano', 'metropolitana-de-recife']

Municipios = ['maceio',
            'marechal-deodoro',
            'rio-largo',
            'belmonte',
            'eunapolis',
            'itamaraju',
            'jucurucu',
            'porto-seguro',
            'prado',
            'santa-cruz-cabralia',
            'teixeira-de-freitas',
            'abreu-e-lima',
            'cabo-de-santo-agostinho',
            'igarassu',
            'jaboatao-dos-guararapes',
            'olinda',
            'paulista',
            'recife']


Unids_Territoriais = [Estados, MesoRegs, MicroRegs, Municipios]
Filterable_cols    = ['nome_UF', 'nome_mesorregiao', 'nome_microrregiao', 'nome_municipio']
Filters            = dict(zip(Filterable_cols, Unids_Territoriais))

# Esvaziando Filtered_DFs e mantendo apenas os DFs a serem processados em Working_DFs
Working_DFs.clear()
Filtered_DFs.clear()
Errors_list.clear()
Working_DFs['DF_Areas_Risco_Domic_Pessoas'] = DF_Areas_Risco_Domic_Pessoas

# Inicia loop iterando pelos dfs em Working_DFs
for df_name in Project_DFs[3].keys(): 
    # Itera pela lista de nomes de colunas definidos como keys em Filters
    for col in Filters.keys(): 
        # Itera pela lista de nomes de unidades territoriais definidos como values em Filters
        for unid in Filters[col]: 
            # Verifica se o filtro existe na coluna em que é aplicado
            if unid not in Project_DFs[3][df_name][col].tolist(): 
                #Adiciona filtros inexistentes em Errors_list
                Errors_list.append(unid)
                print(f'''
                        {unid} não existe em {col} no {df_name}
                        {df_name}_{unid} não foi criado.

                       ''')
            # Aplica make_filtered_df e salva arquivos no save_path
            else: 
                mask = []
                mask.append(unid)
                make_filtered_dfs(df_name, col, mask, 3, True, save_path)



In [ ]:
# Verificando Errors_list
print(Errors_list)
DF_Areas_Risco_Domic_Pessoas[DF_Areas_Risco_Domic_Pessoas['nome_municipio'].isin(Errors_list)]
#* Municipios em Errors_list de fato não constam em DF_Areas_Risco_Domic_Pessoas

In [ ]:
# Cria DFs com todos os municípios, estados e regiões do projeto a partir de DF_Areas_Risco_Domic_Pessoas
DF_Areas_Risco_Municipios    = DF_Areas_Risco_Domic_Pessoas[DF_Areas_Risco_Domic_Pessoas['nome_municipio'].isin(Municipios)]
DF_Areas_Risco_Estados       = DF_Areas_Risco_Domic_Pessoas[DF_Areas_Risco_Domic_Pessoas['nome_UF'].isin(Estados)]
DF_Areas_Risco_Mesorregiões  = DF_Areas_Risco_Domic_Pessoas[DF_Areas_Risco_Domic_Pessoas['nome_UF'].isin(MesoRegs)]
DF_Areas_Risco_Microrregiões = DF_Areas_Risco_Domic_Pessoas[DF_Areas_Risco_Domic_Pessoas['nome_UF'].isin(MicroRegs)]

# Atualiza Filtered_DFs conforme necessário
dict_update_line = {
                   'DF_Areas_Risco_Municipios'   : DF_Areas_Risco_Domic_Pessoas,
                   'DF_Areas_Risco_Estados'      : DF_Areas_Risco_Domic_Pessoas,
                   'DF_Areas_Risco_Mesorregiões' : DF_Areas_Risco_Domic_Pessoas,
                   'DF_Areas_Risco_Microrregiões': DF_Areas_Risco_Domic_Pessoas
                   }

Filtered_DFs.update(dict_update_line)

# Exporta DFs filtrados em .xlsx
for key in list(dict_update_line.keys()):
    dict_update_line[key].to_excel(f'{save_path}\\{key}.xlsx', index = False)




## Tratando raw_Mapa_INSAN

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Mapa_INSAN = Raw_DFs['raw_Mapa_INSAN_Raw'].copy()
Working_DFs.clear()
Working_DFs['df_Mapa_INSAN'] = df_Mapa_INSAN
df = df_Mapa_INSAN

# Remove \n de nomes de colunas
df.rename(columns= lambda s: ''.join(s.split('\n')), inplace=True)
df.rename(columns= lambda s: s.replace('%', ' %').replace('  ',' '), inplace=True)

# Aplica padrão do projeto com add_IBGE_std( ) a partir da coluna 'Código Município"
df = add_IBGE_std(df, 'Código Município IBGE', 'all', True, ['UF', 'Código Município IBGE', 'Nome do Município'])

# Checando valores nulos
DF_Mapa_INSAN.isnull().sum()
#* Não há nulos

# Adiciona DF tratado em Treated_DFs
DF_Mapa_INSAN                = df
Treated_DFs['DF_Mapa_INSAN'] = DF_Mapa_INSAN

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\INSAN'
DF_Mapa_INSAN.to_excel(f'{save_path}\\DF_Mapa_INSAN.xlsx', index = False)



In [ ]:
# Cria DFs filtrados por municípios e estados do projeto
DF_Mapa_INSAN_Municipios    = DF_Mapa_INSAN[DF_Mapa_INSAN['nome_municipio'].isin(Municipios)]
DF_Mapa_INSAN_Estados       = DF_Mapa_INSAN[DF_Mapa_INSAN['nome_UF'].isin(Estados)]
DF_Mapa_INSAN_Mesorregiões  = DF_Mapa_INSAN[DF_Mapa_INSAN['nome_mesorregiao'].isin(MesoRegs)]
DF_Mapa_INSAN_Microrregiões = DF_Mapa_INSAN[DF_Mapa_INSAN['nome_microrregiao'].isin(MicroRegs)]

# Salvando dfs filtrados nos dicionários apropriados
Filtered_DFs['DF_Mapa_INSAN_Municipios'] = DF_Mapa_INSAN_Municipios
Filtered_DFs['DF_Mapa_INSAN_Estados']    = DF_Mapa_INSAN_Estados

# Atualiza Filtered_DFs conforme necessário
dict_update_line = {
                   'DF_Mapa_INSAN_Municipios'   : DF_Mapa_INSAN_Municipios,
                   'DF_Mapa_INSAN_Estados'      : DF_Mapa_INSAN_Estados,
                   'DF_Mapa_INSAN_Mesorregiões' : DF_Mapa_INSAN_Mesorregiões,
                   'DF_Mapa_INSAN_Microrregiões': DF_Mapa_INSAN_Microrregiões
                   }

Filtered_DFs.update(dict_update_line)

# Exporta DFs filtrados em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\INSAN'
for key in list(dict_update_line.keys()):
    dict_update_line[key].to_excel(f'{save_path}\\{key}.xlsx', index = False)


## Tratando planilhas organizadas no Excel a partir de bases de dados online

### Tratando raw_População

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_População                = Raw_DFs['raw_População'].copy()
Working_DFs['df_População'] = df_População
df                          = df_População

# Aplica padrão do projeto a partir dos nomes de municípios
df = add_IBGE_std(df, 'Nome do Município', 'all', False, ['Nome da UF',	'Nome do Município', 'Ano'])

# Checando nulos
df.isnull().sum()
#* Necessário remover colunas vazias.
#* O excesso de nulos se dá pela tabela possuir as colunas Cor, Sexo 
#* e Situação de Domicílio como colunas de filtro (repetindo os dados 3x por cada categoria em cada coluna)

# Remove colunas vazias
df.drop(['Unnamed: 26', 'Unnamed: 27'], axis = 1, inplace = True)

# Checando coluna nome_municipio
#* Dois nulos convertidos em str values
df.drop([468, 469], inplace = True)

# Armazena df_População em DF_População_Municipios em Treated_DFs
DF_População_Municipios                = df
Treated_DFs['DF_População_Municipios'] = DF_População_Municipios

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\População'
DF_População_Municipios.to_excel(f'{save_path}\\DF_População_Municipios.xlsx', index = False)


### Tratando raw_Educação

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Educação                = Raw_DFs['raw_Educação'].copy()
Working_DFs['df_Educação'] = df_Educação
df                         = df_Educação

# Checando nulos
df.isnull().sum()
NaN_rows(df, 'AA')
#* Necessário remover linhas vazias e irrelevantes

# Remove linhas vazias
df.drop([18,19], inplace = True)

#Checa nulos novamente
df.isnull().sum()
#* 0 nulos

# Aplica padrão do projeto a partir dos nomes de municípios
df = add_IBGE_std(df, 'AA.1', 'all', False, ['AA', 'AA.1'])

# Armazenando df_Educação em Treated_DFs
DF_Educação_Municipios                = df
Treated_DFs['DF_Educação_Municipios'] = DF_Educação_Municipios

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Educação'
DF_Educação_Municipios.to_excel(f'{save_path}\\DF_Educação_Municipios.xlsx', index = False)


### Tratando raw_Renda_e_Ocupação

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Renda_e_Ocupação                = Raw_DFs['raw_Renda_e_Ocupação'].copy()
Working_DFs['df_Renda_e_Ocupação'] = df_Renda_e_Ocupação
df                                 = df_Renda_e_Ocupação

# Checando nulos
df.isnull().sum()
NaN_rows(df, 'Nome do Município')
#* Necessário remover linhas vazias e irrelevantes em Nome do Município

# Remove linhas vazias
df.drop([468,469], inplace = True)

df.isnull().sum()
# #* 0 nulos

# Aplica padrão do projeto a partir dos nomes de municípios
df = add_IBGE_std(df, 'Nome do Município', 'mun', False, ['Nome da UF', 'Nome do Município', 'Ano'])

# Armazenando DF_Renda_e_Ocupação em Treated_DFs
DF_Renda_e_Ocupação_Municipios                = df
Treated_DFs['DF_Renda_e_Ocupação_Municipios'] = DF_Renda_e_Ocupação_Municipios

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Renda e Ocupação'
DF_Renda_e_Ocupação_Municipios.to_excel(f'{save_path}\\DF_Renda_e_Ocupação_Municipios.xlsx', index = False)


### Tratando raw_Pobreza_e_Vulnerabilidade

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Pobreza_e_Vulnerabilidade                = Raw_DFs['raw_Pobreza_e_Vulnerabilidade'].copy()
Working_DFs['df_Pobreza_e_Vulnerabilidade'] = df_Pobreza_e_Vulnerabilidade
df                                          = df_Pobreza_e_Vulnerabilidade

# Checando nulos
df.isnull().sum()
#* Grande quantidade de nulos devido às colunas Cor, Sexo e Situação de Domicílio, que 
#* repetem os dados e possuem alguns ausentespara cada categoria que possuem 

NaN_rows(df, 'Nome do Município')
#* Necessário remover linhas vazias e irrelevantes em Nome do Município

# Remove linhas vazias
df.drop([468,469], inplace = True)

df.isnull().sum()
#* 0 nulos em Nome do Município

# Aplica padrão do projeto a partir dos nomes de municípios
df = add_IBGE_std(df, 'Nome do Município', 'mun', False, ['Nome da UF', 'Nome do Município', 'Ano'])

# # Armazenando DF_Renda_e_Ocupação em Treated_DFs
DF_Pobreza_e_Vulnerabilidade_Municipios                = df
Treated_DFs['DF_Pobreza_e_Vulnerabilidade_Municipios'] = DF_Pobreza_e_Vulnerabilidade_Municipios

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Pobreza e Vulnerabilidade'
DF_Pobreza_e_Vulnerabilidade_Municipios.to_excel(f'{save_path}\\DF_Pobreza_e_Vulnerabilidade_Municipios.xlsx', index = False)


### Tratando raw_Aglomerados_Subnormais

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Aglomerados_Subnormais                = Raw_DFs['raw_Aglomerados_Subnormais'].copy()
Working_DFs['df_Aglomerados_Subnormais'] = df_Aglomerados_Subnormais
df                                       = df_Aglomerados_Subnormais

# Checando nulos
df.isnull().sum()
NaN_rows(df, 'NM_MUN')
#* Necessário remover linhas vazias e irrelevantes em Nome do Município

# Remove linhas vazias
df.drop([18,19], inplace = True)

df.isnull().sum()
#* 0 nulos em Nome do Município

# Aplica padrão do projeto a partir dos nomes de municípios
df = add_IBGE_std(df, 'NM_MUN', 'all', False, ['OBJECTID',	'CD_MUN',	'UF',	'NM_MUN'])

DF_Aglomerados_Subnormais_Municipios                = df
Treated_DFs['DF_Aglomerados_Subnormais_Municipios'] = DF_Aglomerados_Subnormais_Municipios

# Exporta DF tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\AGSN'
DF_Aglomerados_Subnormais_Municipios.to_excel(f'{save_path}\\DF_Aglomerados_Subnormais_Municipios.xlsx', index = False)


## Tratando e unindo dfs sobre pobreza 1991-2010

### Tratando raw_%_Pobres_Municipal_BR_1991_2010

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Pobres_Municipal_BR_1991_2010                = Raw_DFs['raw_%_Pobres_Municipal_BR_1991_2010'].copy()
Working_DFs['df_Prop_Pobres_Municipal_BR_1991_2010'] = df_Prop_Pobres_Municipal_BR_1991_2010
df                                                   = df_Prop_Pobres_Municipal_BR_1991_2010

# Checando nulos
df.isnull().sum()
#* 0 nulos em Nome do Município
#* nulos em todas as colunas com dados, municípios sem dados, nenhum na lista do projeto
NaN_rows(df, '1991')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices do df resultante em lista
excluded_rows = NaN_rows(df, '1991').index.tolist()
excluded_rows

# Exclui lista de linhas com nulos
df.drop(excluded_rows, inplace = True)

df.isnull().sum()
#* 0 nulos no df

# Aplica padrão do projeto a partir de coluna com códigos IBGE
df = add_IBGE_std(df, 'Código', 'all', True, ['Código','Município', 'Sigla'])

# Renomeia colunas que serão repetidas no merge
df.rename(columns={
                  '1991': '% Pobres 1991',
                  '2000': '% Pobres 2000',
                  '2010': '% Pobres 2010' }, inplace = True)

# Reserva dataframe para realizar join posteriormente
df_left = df


### Limpando df_Prop_Extremam_Pobres_Municipal_BR_1991_2010

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Extrem_Pobres_Municipal_BR_1991_2010                = Raw_DFs['raw_%_de_Extremamente_Pobres_Municipal_BR_1991_2010'].copy()
Working_DFs['df_Prop_Extrem_Pobres_Municipal_BR_1991_2010'] = df_Prop_Extrem_Pobres_Municipal_BR_1991_2010
df                                                          = df_Prop_Extrem_Pobres_Municipal_BR_1991_2010

# Checando nulos
df.isnull().sum()
#* 0 nulos em Nome do Município e 32 em todas as colunas, municipios sem dados
NaN_rows(df, '1991')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices do df resultante em lista
excluded_rows = NaN_rows(df, '1991').index.tolist()
excluded_rows

# Exclui lista de linhas com nulos
df.drop(excluded_rows, inplace = True)

# Aplica padrão do projeto a partir de coluna com códigos IBGE
df = add_IBGE_std(df, 'Código', 'all', True, ['Código','Município', 'Sigla'])

# Renomeia colunas que serão repetidas no merge
df.rename(columns={
                  '1991': '% Ext Pobres 1991',
                  '2000': '% Ext Pobres 2000',
                  '2010': '% Ext Pobres 2010' }, inplace = True)

# Reserva dataframe para realizar join posteriormente
df_right = df


### Unindo df_Prop_Pobres_Municipal_BR_1991_2010 e df_Prop_Extrem_Pobres_Municipal_BR_1991_2010

In [ ]:
# Faz inner join usando cod_municipio como key e altera sufixos para facilitar remoção de duplicatas
DF_Prop_Pobreza_Municipios_91_10 = pd.merge(df_left,
                                            df_right, 
                                            how      = 'inner',
                                            on       = 'nome_municipio',
                                            suffixes = ('', '_remove'))

# Deleta colunas excedentes criadas pelo merge()
DF_Prop_Pobreza_Municipios_91_10.drop([col for col in DF_Prop_Pobreza_Municipios_91_10.columns if 'remove' in col], axis = 1, inplace = True)

Treated_DFs['DF_Prop_Pobreza_Municipios_91_10'] = DF_Prop_Pobreza_Municipios_91_10

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Pobreza e Vulnerabilidade'
DF_Prop_Pobreza_Municipios_91_10.to_excel(f'{save_path}\\DF_Prop_Pobreza_Municipios_91_10.xlsx', index = False)



## Tratando dfs PIB 1970-2020

### Tratando 'raw_PIB_Municipal_1970'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_PIB_Municipal_1970                = Raw_DFs['raw_PIB_Municipal_1970'].copy()
Working_DFs['df_PIB_Municipal_1970'] = df_PIB_Municipal_1970
df                                   = df_PIB_Municipal_1970


df.isnull().sum()
#* 0 nulos em Nome do Município e 1645 nulos idênticos em todas as colunas, municipios sem dados
NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (1970)')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices desse df em lista
excluded_rows = NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (1970)').index.tolist()
df.drop(excluded_rows, inplace=True)
#* 3951 linhas permanecem

# Exclui coluna vazia
df.drop(['Unnamed: 7'], axis = 1, inplace = True)

df.isnull().sum()
#* 0 nulos no df

df = add_IBGE_std(df, 'Código', 'mun', True, ['Sigla',	'Código',	'Município'])

# Checando municipios dos projetos
df[df['nome_municipio'].isin(Municipios)]
#*1 municipio homôninmo não pertinente

# transforma valor da célula específica
df.at[2711, 'nome_municipio' ] = 'paulista-pb'

DF_PIB_Municipal_1970 = df


### Tratando 'raw_PIB_Municipal_1996'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_PIB_Municipal_1996                = Raw_DFs['raw_PIB_Municipal_1996'].copy()
Working_DFs['df_PIB_Municipal_1996'] = df_PIB_Municipal_1996
df                                   = df_PIB_Municipal_1996


# Checando nulos
df.isnull().sum()
# #* 0 nulos em Nome do Município e 622 nulos idênticos em todas as colunas, municipios sem dados
NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (1996)')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices desse df em lista
excluded_rows = NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (1996)').index.tolist()
df.drop(excluded_rows, inplace=True)
#* 4974 linhas permanecem

# Exclui coluna vazia
df.drop(['Unnamed: 7'], axis = 1, inplace = True)

df.isnull().sum()
# #* 0 nulos no df

# Aplica padrão do projeto usando a coluna código IBGE como base
df = add_IBGE_std(df, 'Código', 'mun', True, ['Sigla',	'Código',	'Município'])
df

# Checando municipios dos projetos
df[df['nome_municipio'].isin(Municipios)]
#*2 municipios homôninmos não pertinentes

# transforma valores das células específicas
df.at[2711, 'nome_municipio' ] = 'paulista-pb'
df.at[4473, 'nome_municipio' ] = 'belmonte-sc'

DF_PIB_Municipal_1996 = df


### Tratando  raw_PIB_Municipal_2000

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_PIB_Municipal_2000                = Raw_DFs['raw_PIB_Municipal_2000'].copy()
Working_DFs['df_PIB_Municipal_2000'] = df_PIB_Municipal_2000
df                                   = df_PIB_Municipal_2000

# Checando nulos
df.isnull().sum()
# #* 0 nulos em Nome do Município e 89 nulos idênticos em todas as colunas, municipios sem dados
NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (2000)')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices desse df em lista
excluded_rows = NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (2000)').index.tolist()
df.drop(excluded_rows, inplace=True)
df.isnull().sum()

# Exclui coluna vazia
df.drop(['Unnamed: 7'], axis = 1, inplace = True)

df.isnull().sum()
# #* 0 nulos no df

# Aplica padrão do projeto usando a coluna código IBGE como base
df = add_IBGE_std(df, 'Código', 'mun', True, ['Sigla',	'Código',	'Município'])

# Checando municipios dos projetos
df[df['nome_municipio'].isin(Municipios)]
# #*2 municipios homôninmos não pertinentes

# transforma valores das células específicas
df.at[2711, 'nome_municipio' ] = 'paulista-pb'
df.at[4473, 'nome_municipio' ] = 'belmonte-sc'

DF_PIB_Municipal_2000 = df


### Tratando  raw_PIB_Municipal_2010

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_PIB_Municipal_2010                = Raw_DFs['raw_PIB_Municipal_2010'].copy()
Working_DFs['df_PIB_Municipal_1996'] = df_PIB_Municipal_2010
df                                   = df_PIB_Municipal_2010

# Checando nulos
df.isnull().sum()
# #* 0 nulos em Nome do Município e 32 nulos idênticos em todas as colunas, municipios sem dados
NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (2010)')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices desse df em lista
excluded_rows = NaN_rows(df, 'PIB Municipal - preços de mercado (preços de 2010) (2010)').index.tolist()
df.drop(excluded_rows, inplace=True)
#* 5564 linhas permanecem

# Exclui coluna vazia
df.drop(['Unnamed: 7'], axis = 1, inplace = True)

df.isnull().sum()
# #* 0 nulos no df

# Aplica padrão do projeto usando a coluna código IBGE como base
df = add_IBGE_std(df, 'Código', 'mun', True, ['Sigla',	'Código',	'Município'])

# Checando municipios dos projetos
df[df['nome_municipio'].isin(Municipios)]
# #*2 municipios homôninmos não pertinentes

# transforma valores das células específicas
df.at[2711, 'nome_municipio' ] = 'paulista-pb'
df.at[4473, 'nome_municipio' ] = 'belmonte-sc'

DF_PIB_Municipal_2010 = df


### Tratando  raw_PIB_Municipal_2020

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_PIB_Municipal_2020                = Raw_DFs['raw_PIB_Municipal_2020'].copy()
Working_DFs['df_PIB_Municipal_2020'] = df_PIB_Municipal_2020
df                                   = df_PIB_Municipal_2020

# Checando nulos
df.isnull().sum()
# #* 0 nulos em Nome do Município e 32 nulos idênticos em todas as colunas, municipios sem dados
NaN_rows(df, 'PIB Municipal (valor adicionado a preços básicos) - indústria (preços de 2010) (2020)')

# Cria máscara para filtrar linhas sem nulos e adiciona os índices desse df em lista
excluded_rows = NaN_rows(df, 'PIB Municipal (valor adicionado a preços básicos) - indústria (preços de 2010) (2020)').index.tolist()
df.drop(excluded_rows, inplace=True)
#* 5564 linhas permanecem

# Exclui colunas vazias
df.drop(['Unnamed: 7'], axis = 1, inplace = True)
df.drop(['PIB Municipal - preços de mercado (preços de 2010) (2020)'], axis = 1, inplace = True)

df.isnull().sum()
# #* 0 nulos no df

# Aplica padrão do projeto usando a coluna código IBGE como base
df = add_IBGE_std(df, 'Código', 'mun', True, ['Sigla',	'Código',	'Município'])

# Checando municipios dos projetos
df[df['nome_municipio'].isin(Municipios)]
#*2 municipios homôninmos não pertinentes

# transforma valores das células específicas
df.at[2711, 'nome_municipio' ] = 'paulista-pb'
df.at[4473, 'nome_municipio' ] = 'belmonte-sc'

DF_PIB_Municipal_2020 = df


### Unindo dfs PIB_Municipal 1970, 1996, 2000, 2010, 2020

In [ ]:
# Reduz nomes dos dfs
A = DF_PIB_Municipal_1970.copy()
B = DF_PIB_Municipal_1996.copy()
C = DF_PIB_Municipal_2000.copy()
D = DF_PIB_Municipal_2010.copy()
E = DF_PIB_Municipal_2020.copy()


# Faz inner join usando cod_municipio como key e altera sufixos para facilitar remoção de duplicatas
AB = pd.merge(
            A,
            B, 
            how      = 'right',
            on       = 'cod_municipio',
            suffixes = ('', '_remove'))
AB.drop([col for col in AB.columns if 'remove' in col], axis = 1, inplace = True)

ABC = pd.merge(
            AB,
            C, 
            how      = 'right',
            on       = 'cod_municipio',
            suffixes = ('', '_remove'))
ABC.drop([col for col in ABC.columns if 'remove' in col], axis = 1, inplace = True)

ABCD = pd.merge(
            ABC,
            D, 
            how      = 'right',
            on       = 'cod_municipio',
            suffixes = ('', '_remove'))
ABCD.drop([col for col in ABCD.columns if 'remove' in col], axis = 1, inplace = True)

ABCDE = pd.merge(
            ABCD,
            E, 
            how      = 'right',
            on       = 'cod_municipio',
            suffixes = ('', '_remove'))
ABCDE.drop([col for col in ABCDE.columns if 'remove' in col], axis = 1, inplace = True)

DF_PIB_Municipal_1970_2020 = ABCDE
Treated_DFs['DF_PIB_Municipal_1970_2020'] = DF_PIB_Municipal_1970_2020

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\PIB'
DF_PIB_Municipal_1970_2020.to_excel(f'{save_path}\\DF_PIB_Municipal_1970_2020.xlsx', index = False)


In [ ]:
# Cria DFs com todos os municípios e estados do projeto a partir de DF_Areas_Risco_Domic_Pessoas
DF_PIB_Municipal_1970_2020_Municipios    = DF_PIB_Municipal_1970_2020[DF_PIB_Municipal_1970_2020['nome_municipio'].isin(Municipios)]
DF_PIB_Municipal_1970_2020_Estados       = DF_PIB_Municipal_1970_2020[DF_PIB_Municipal_1970_2020['nome_UF'].isin(Estados)]

# Atualiza Filtered_DFs conforme necessário
dict_update_line = {
                   'DF_PIB_Municipal_1970_2020_Municipios': DF_PIB_Municipal_1970_2020_Municipios,
                   'DF_PIB_Municipal_1970_2020_Estados'   : DF_PIB_Municipal_1970_2020_Estados
                   }

Filtered_DFs.update(dict_update_line)

# Exporta DFs filtrados em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\PIB'
for key in list(dict_update_line.keys()):
    dict_update_line[key].to_excel(f'{save_path}\\{key}.xlsx', index = False)




## Tratando dfs AL-BA-PE

 ### Tratando raw_%_Pobres_-_linha_de_pobreza_baseada_em_necessidades_calóricas_-_Estadual_BR_1976-2014_Pobres___linha_de_pobreza_baseada_em_necessidades_calóricas___Estadual_BR_1976_2014

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Abaixo_Linha_de_Pobreza_Kcal_91_10                = Raw_DFs['raw_%_Pobres___linha_de_pobreza_baseada_em_necessidades_calóricas___Estadual_BR_1976_2014'].copy()
Working_DFs['df_Abaixo_Linha_de_Pobreza_Kcal_91_10'] = df_Abaixo_Linha_de_Pobreza_Kcal_91_10
df                                                   = df_Abaixo_Linha_de_Pobreza_Kcal_91_10

#Checa existência de valores nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls

# Aplica padronização IBGE para cods. e nomes das unidades da federação
df = add_IBGE_std(df, 'Código', 'UF', True, ['Sigla',	'Código',	'Estado'])

DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010 = df
Treated_DFs['DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010'] = DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Pobreza e Vulnerabilidade'
DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010.to_excel(f'{save_path}\\DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010.xlsx', index = False)

# filtrando DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010 para alagoas, bahia e pernambuco utilizando a função make_filtered_dfs()
Working_DFs.clear()
Working_DFs['DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010'] = DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010
make_filtered_dfs('DF_Abaixo_Linha_de_Pobreza_Kcal_91_2010', 'nome_UF', ['alagoas', 'bahia', 'pernambuco'], 3, True, save_path)


dict_keys(['DF_Areas_Risco_Domic_Pessoas_alagoas', 'DF_Areas_Risco_Domic_Pessoas_bahia', 'DF_Areas_Risco_Domic_Pessoas_pernambuco', 'DF_Areas_Risco_Domic_Pessoas_leste_alagoano', 'DF_Areas_Risco_Domic_Pessoas_sul_baiano', 'DF_Areas_Risco_Domic_Pessoas_metropolitana_de_recife', 'DF_Areas_Risco_Domic_Pessoas_maceio', 'DF_Areas_Risco_Domic_Pessoas_itamaraca', 'DF_Areas_Risco_Domic_Pessoas_recife', 'DF_Areas_Risco_Domic_Pessoas_suape', 'DF_Areas_Risco_Domic_Pessoas_porto_seguro', 'DF_Areas_Risco_Domic_Pessoas_ilheus_itabuna', 'DF_Areas_Risco_Domic_Pessoas_marechal_deodoro', 'DF_Areas_Risco_Domic_Pessoas_belmonte', 'DF_Areas_Risco_Domic_Pessoas_itamaraju', 'DF_Areas_Risco_Domic_Pessoas_prado', 'DF_Areas_Risco_Domic_Pessoas_santa_cruz_cabralia', 'DF_Areas_Risco_Domic_Pessoas_abreu_e_lima', 'DF_Areas_Risco_Domic_Pessoas_cabo_de_santo_agostinho', 'DF_Areas_Risco_Domic_Pessoas_igarassu', 'DF_Areas_Risco_Domic_Pessoas_jaboatao_dos_guararapes', 'DF_Areas_Risco_Domic_Pessoas_olinda', 'DF_Areas_Ris

### Tratando 'raw_%_Crianças_Pobres_Censos_(91_2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Crianças_Pobres_91_10_AL_BA_PE                = Raw_DFs['raw_%_Crianças_Pobres_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Prop_Crianças_Pobres_91_10_AL_BA_PE'] = df_Prop_Crianças_Pobres_91_10_AL_BA_PE
df                                                    = df_Prop_Crianças_Pobres_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Deletando linhas vazias ou irrelevantes
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
#! importar fnmatch e re (instalados na célula de preparação do ambiente)
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Prop_Crianças_Pobres_91_10_AL_BA_PE                = df
Treated_DFs['DF_Prop_Crianças_Pobres_91_10_AL_BA_PE'] = DF_Prop_Crianças_Pobres_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Pobreza e Vulnerabilidade'
DF_Prop_Crianças_Pobres_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Prop_Crianças_Pobres_91_10_AL_BA_PE.xlsx', index = False)



C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\3853510457.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\3853510457.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])


### Tratando 'raw_%_Pobreza_e_Renda_Pobreza_Municípios_Censos_(91_2010)_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Pobreza_Renda_91_10_AL_BA_PE                = Raw_DFs['raw_%_Pobreza_e_Renda_Pobreza_Municípios_Censos_(91_2010)_AL_BA_PE'].copy()
Working_DFs['df_Prop_Pobreza_Renda_91_10_AL_BA_PE'] = df_Prop_Pobreza_Renda_91_10_AL_BA_PE
df                                                  = df_Prop_Pobreza_Renda_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Prop_Pobreza_Renda_91_10_AL_BA_PE                = df
Treated_DFs['DF_Prop_Pobreza_Renda_91_10_AL_BA_PE'] = DF_Prop_Pobreza_Renda_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Pobreza e Vulnerabilidade'
DF_Prop_Pobreza_Renda_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Prop_Pobreza_Renda_91_10_AL_BA_PE.xlsx', index = False)


C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\491026584.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\491026584.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])


### Tratando 'raw_Dados_Renda_Detalhados_Censos_(91_2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Renda_Detalhada_91_10_AL_BA_PE                = Raw_DFs['raw_Dados_Renda_Detalhados_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Renda_Detalhada_91_10_AL_BA_PE'] = df_Renda_Detalhada_91_10_AL_BA_PE
df                                               = df_Renda_Detalhada_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação e Rendimento médio dos ocupados 1991 e	Rendimento médio dos ocupados 2000
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

#deletando colunas
df.drop(['Rendimento médio dos ocupados 1991','Rendimento médio dos ocupados 2000'], axis = 1, inplace=True)

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])


DF_Renda_Detalhada_91_10_AL_BA_PE                = df
Treated_DFs['DF_Renda_Detalhada_91_10_AL_BA_PE'] = DF_Renda_Detalhada_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Renda e Ocupação'
DF_Renda_Detalhada_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Renda_Detalhada_91_10_AL_BA_PE.xlsx', index = False)



C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\2210041813.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Rendimento médio dos ocupados 1991','Rendimento médio dos ocupados 2000'], axis = 1, inplace=True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\2210041813.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\2210041813.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

### Tratando 'raw_Renda_Domiciliar_Estratificada_Censos_(91_2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Renda_Domiciliar_Estrat_91_10_AL_BA_PE                = Raw_DFs['raw_Renda_Domiciliar_Estratificada_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Renda_Domiciliar_Estrat_91_10_AL_BA_PE'] = df_Renda_Domiciliar_Estrat_91_10_AL_BA_PE
df                                                       = df_Renda_Domiciliar_Estrat_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Renda_Domiciliar_Estrat_91_10_AL_BA_PE                = df
Treated_DFs['DF_Renda_Domiciliar_Estrat_91_10_AL_BA_PE'] = DF_Renda_Domiciliar_Estrat_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Renda e Ocupação'
DF_Renda_Domiciliar_Estrat_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Renda_Domiciliar_Estrat_91_10_AL_BA_PE.xlsx', index = False)


C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\3017711735.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\3017711735.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])


### Tratando 'raw_Estratificação_Renda_Censos_(91_2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Estratificação_Renda_91_10_AL_BA_PE                = Raw_DFs['raw_Estratificação_Renda_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Estratificação_Renda_91_10_AL_BA_PE'] = df_Estratificação_Renda_91_10_AL_BA_PE
df                                                  = df_Estratificação_Renda_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação, % dos ocupados sem rendimento 1991, % dos ocupados com rendimento de até 1 salário mínimo 1991
#* % dos ocupados com rendimento de até 2 salários mínimo 1991, % dos ocupados com rendimento de até 3 salários mínimo 1991, % dos ocupados com rendimento de até 5 salários mínimo 1991
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Seleciona todas as colunas que NÃO possuem com as substrings: 'mínimo 1991' e 'rendimento 1991'
df = df.loc[:,~df.columns.str.contains('mínimo 1991')]
df = df.loc[:,~df.columns.str.contains('rendimento 1991')]

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Estratificação_Renda_91_10_AL_BA_PE                = df
Treated_DFs['DF_Estratificação_Renda_91_10_AL_BA_PE'] = DF_Estratificação_Renda_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Renda e Ocupação'
DF_Estratificação_Renda_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Estratificação_Renda_91_10_AL_BA_PE.xlsx', index = False)

### Tratando 'raw_%_População_com_acesso_a_infraestrutura_Censos_(91_2010)_Municipios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE                = Raw_DFs['raw_%_População_com_acesso_a_infraestrutura_Censos_(91_2010)_Municipios_AL_BA_PE'].copy()
Working_DFs['df_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE'] = df_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE
df                                                           = df_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Seleciona todas as colunas que NÃO possuem as substrings 'aparelhada 1991' ou 'aparelhada 2000'
df = df.loc[:,~df.columns.str.contains('aparelhada 1991')]
df = df.loc[:,~df.columns.str.contains('aparelhada 2000')]

#eliminando linhas
df.drop([0, 705,706,707], inplace = True)

#padronizando nomes de municípios fora do padrão IBGE
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE                = df
Treated_DFs['DF_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE'] = DF_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Infraestrutura Urbana'
DF_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Prop_Pop_Sem_Infraestrutura_91_10_AL_BA_PE.xlsx', index = False)


### Tratando 'raw_%_População_com_acesso_a_saneamento_SNIS_(2017)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE                = Raw_DFs['raw_%_População_com_acesso_a_saneamento_SNIS_(2017)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE'] = df_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE
df                                                       = df_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE                = df
Treated_DFs['DF_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE'] = DF_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Infraestrutura Urbana'
DF_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Prop_Pop_Sem_Saneamento_SNIS2017_91_10_AL_BA_PE.xlsx', index = False)


C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\2639328626.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\2639328626.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])


### Tratando 'raw_%_População_vivendo_em_domicílios_precários_Censos_(91_2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE                = Raw_DFs['raw_%_População_vivendo_em_domicílios_precários_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE'] = df_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE
df                                                       = df_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Seleciona todas as colunas que NÃO possuem com as substrings: 'aparelhada 1991' e 'aparealhada 2000'
df = df.loc[:,~df.columns.str.contains('aparelhada 1991')]
df = df.loc[:,~df.columns.str.contains('aparelhada 2000')]


# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE                = df
Treated_DFs['DF_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE'] = DF_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Infraestrutura Urbana'
DF_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Prop_Pop_Domicílios_Precários_91_10_AL_BA_PE.xlsx', index = False)


### Tratando 'raw_Dados_Educação_Censos_(1991_2010)_por_Sexo,_Cor_e_Modo_de_Vida_Municípios_AL_BA_PE'

In [ ]:
Raw_DFs['raw_Dados_Educação_Censos_(1991_2010)_por_Sexo,_Cor_e_Modo_de_Vida_Municípios_AL_BA_PE'].isnull().sum()

Territorialidades                                                                           0
% de 18 a 20 anos de idade com ensino médio completo 1991                                  50
% de 18 a 20 anos de idade com ensino médio completo 2000                                  12
% de 18 a 20 anos de idade com ensino médio completo 2010                                   3
% de 15 a 17 anos de idade com ensino fundamental completo 1991                            22
                                                                                         ... 
Desagregação URBANO % de 25 anos ou mais de idade com ensino superior completo Censo.1    708
Desagregação URBANO % de 25 anos ou mais de idade com ensino superior completo Censo.2    707
Desagregação URBANO Expectativa de anos de estudo aos 18 anos de idade Censo              708
Desagregação URBANO Expectativa de anos de estudo aos 18 anos de idade Censo.1            708
Desagregação URBANO Expectativa de anos de estudo aos 18 ano

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Dados_Educação_Censo_91_10_AL_BA_PE                = Raw_DFs['raw_Dados_Educação_Censos_(1991_2010)_por_Sexo,_Cor_e_Modo_de_Vida_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Dados_Educação_Censo_91_10_AL_BA_PE'] = df_Dados_Educação_Censo_91_10_AL_BA_PE
df                                                  = df_Dados_Educação_Censo_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação, % dos ocupados sem rendimento 1991, % dos ocupados com rendimento de até 1 salário mínimo 1991
#* % dos ocupados com rendimento de até 2 salários mínimo 1991, % dos ocupados com rendimento de até 3 salários mínimo 1991, % dos ocupados com rendimento de até 5 salários mínimo 1991
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)

# Seleciona todas as colunas que NÃO iniciam com a substring 'Desagregação'
df = df.loc[:,~df.columns.str.contains('^Desagregação')]

# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Dados_Educação_Censo_91_10_AL_BA_PE                = df
Treated_DFs['DF_Dados_Educação_Censo_91_10_AL_BA_PE'] = DF_Dados_Educação_Censo_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Educação'
DF_Dados_Educação_Censo_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Dados_Educação_Censo_91_10_AL_BA_PE.xlsx', index = False)

C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\4131214376.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([0, 705,706,707], inplace = True)
C:\Users\João Flavio\AppData\Local\Temp\ipykernel_20200\4131214376.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])


### Tratando  'raw_Medidas_de_Desigualdade_Censos_(91-2010)_Municípios_AL_BA_PE'

In [ ]:
# Armazena raw_df em cópia de trabalho, esvazia e atualiza Working_DFs
df_Medidas_Desigualdade_Censo_91_10_AL_BA_PE                = Raw_DFs['raw_Medidas_de_Desigualdade_Censos_(91_2010)_Municípios_AL_BA_PE'].copy()
Working_DFs['df_Medidas_Desigualdade_Censo_91_10_AL_BA_PE'] = df_Medidas_Desigualdade_Censo_91_10_AL_BA_PE
df                                                  = df_Medidas_Desigualdade_Censo_91_10_AL_BA_PE

# Checando nulos
df.isnull().sum()
#* DF sem necessidade de dropar Nulls em nomes de municípios
#* Dropar todas as colunas Desagregação, % dos ocupados sem rendimento 1991, % dos ocupados com rendimento de até 1 salário mínimo 1991
#* % dos ocupados com rendimento de até 2 salários mínimo 1991, % dos ocupados com rendimento de até 3 salários mínimo 1991, % dos ocupados com rendimento de até 5 salários mínimo 1991
#* Dropar linhas vazias e irrelevantes (brasil, vazias e fontes)


# Deletando linhas
df.drop([0, 705,706,707], inplace = True)

# Padronizando nomes de municípios fora do padrão IBGE (impedem o mapping adequado para aplicação do padrão do projeto)
df['Territorialidades'] = df['Territorialidades'].replace(['Iguaraci (PE)', 'Muquém de São Francisco (BA)'],['Iguaracy', 'Muquém do São Francisco'])

# Padronizando coluna Territorialidades, removendo '(??)' das strings e aplicando slugify, 
# para permitir a aplicação da padronização do projeto com add_IBGE_std()
df = df.replace(to_replace = '\(.*', value = '', regex = True)
df['Territorialidades'] = df['Territorialidades'].apply(slugify)

# Adicionando padrão IBGE usado no projeto
df = add_IBGE_std(df, 'Territorialidades', 'mun', False, ['Territorialidades'])

DF_Medidas_Desigualdade_Censo_91_10_AL_BA_PE                = df
Treated_DFs['DF_Medidas_Desigualdade_Censo_91_10_AL_BA_PE'] = DF_Medidas_Desigualdade_Censo_91_10_AL_BA_PE

# Exporta df tratado em .xlsx
save_path = 'C:\\Users\\João Flavio\\Desktop\\Code\\PyProjects\\Leonard\\Treated Files Repository\\Desigualdade'
DF_Medidas_Desigualdade_Censo_91_10_AL_BA_PE.to_excel(f'{save_path}\\DF_Medidas_Desigualdade_Censo_91_10_AL_BA_PE.xlsx', index = False)
